In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext('local')
spark = SparkSession(sc)

In [60]:
df = spark.read.csv("time_between_events_test.csv", header=True)

In [61]:
df.toPandas()

,userId,sessionId,playlist_id,event_name,timestamp
0,1,1,A,start,1
1,1,1,None,another,2
2,1,1,A,stop,3
3,1,1,B,start,4
4,1,1,None,another,5
5,1,1,C,start,6
6,1,1,C,stop,7


In [62]:

from pyspark.sql import Window
import time
from pyspark.sql.types import StringType
import pyspark.sql.functions as F
from pyspark.sql.types import TimestampType, LongType

In [63]:
#Window.partitionBy(partitioning_column).orderBy(ordering_column).rangeBetween(Window.unboundedPreceding,last_row)

In [64]:
df = df.filter(df['event_name'].isin(['start', 'stop']))

In [65]:
df.toPandas()

,userId,sessionId,playlist_id,event_name,timestamp
0,1,1,A,start,1
1,1,1,A,stop,3
2,1,1,B,start,4
3,1,1,C,start,6
4,1,1,C,stop,7


In [66]:
df = df.withColumn('tmp-playlist-id', F.when((df["event_name"] == 'start'), F.col('playlist_id')).otherwise("0"))
df.toPandas()


,userId,sessionId,playlist_id,event_name,timestamp,tmp-playlist-id
0,1,1,A,start,1,A
1,1,1,A,stop,3,0
2,1,1,B,start,4,B
3,1,1,C,start,6,C
4,1,1,C,stop,7,0


In [67]:
win = Window.partitionBy(['userId', 'sessionId']).orderBy('timestamp')
df = df.withColumn("playlist_time_end", F.lead('timestamp').over(win))
df.toPandas()

,userId,sessionId,playlist_id,event_name,timestamp,tmp-playlist-id,playlist_time_end
0,1,1,A,start,1,A,3
1,1,1,A,stop,3,0,4
2,1,1,B,start,4,B,6
3,1,1,C,start,6,C,7
4,1,1,C,stop,7,0,None


In [68]:
df = df.filter((F.col('tmp-playlist-id') != '0') & (F.col('playlist_time_end').isNotNull()))
df.toPandas()

,userId,sessionId,playlist_id,event_name,timestamp,tmp-playlist-id,playlist_time_end
0,1,1,A,start,1,A,3
1,1,1,B,start,4,B,6
2,1,1,C,start,6,C,7


In [69]:
df = df.withColumn('playlist_duration', F.col('playlist_time_end') - F.col('timestamp'))
df.toPandas()

,userId,sessionId,playlist_id,event_name,timestamp,tmp-playlist-id,playlist_time_end,playlist_duration
0,1,1,A,start,1,A,3,2.0
1,1,1,B,start,4,B,6,2.0
2,1,1,C,start,6,C,7,1.0
